In [6]:
# load required Pyhton modules 

import pandas as pd
import sqlite3

# Read Data Set and Respective Columns
adult_columns = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names', sep=":")

#Columns Name for DataSet
adult_columns.iloc[91:106,].index.tolist()

['>50K, <=50K.',
 'age',
 'workclass',
 'fnlwgt',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country']

In [8]:
# Reorder columns 
adult_names= adult_columns.iloc[91:106,].index.tolist()
adult_names=adult_names[1:]+adult_names[0::-1]
adult_names

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country',
 '>50K, <=50K.']

In [11]:
# Read Data Set annd Apply coumns Names

adult_data=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data' , names=adult_names,header=None , index_col=False)
# Convert columns name to Title case
adult_data.columns=adult_data.columns.str.capitalize().tolist()
adult_data.head(2)

,Age,Workclass,Fnlwgt,Education,Education-num,Marital-status,Occupation,Relationship,Race,Sex,Capital-gain,Capital-loss,Hours-per-week,Native-country,">50k, <=50k."
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K


In [12]:
#Task
#Create a sql db from adult dataset and name it sqladb

connection = sqlite3.connect('sqladb.db') # Create Database
cursor = connection.cursor()


In [13]:
# Create table 

connection.execute('''
    CREATE TABLE IF NOT EXISTS ADULTS (
         Age INTEGER,
         Workclass VARCHAR(20),
         Fnlwgt INTEGER,
         Education VARCHAR(20),
         Education_num INTEGER,         
         Marital_status VARCHAR(30),         
         Occupation VARCHAR(20),
         Relationship VARCHAR(20),
         Race VARCHAR(20),
         Sex VARCHAR(10),
         Capital_gain INTEGER,
         Capital_loss INTEGER,
         Hours_per_week INTEGER,
         Native_country VARCHAR(30),
         '>50k, <=50k.' VARCHAR(10)
        
    )
''')

connection.commit()

In [14]:
# Insert Data into adult table from  adult_data(dataframe)

insert_query = "INSERT INTO ADULTS (Age, Workclass, Fnlwgt, Education, Education_num, Marital_status, Occupation, Relationship, Race, Sex, Capital_gain, Capital_loss, Hours_per_week, Native_country, '>50k, <=50k.') values (%d,'%s', %d, '%s', %d, '%s','%s','%s','%s','%s',%d,%d,%d,'%s','%s')"

for index, row in adult_data.iterrows():
    connection.execute(insert_query % (row['Age'], row['Workclass'], row['Fnlwgt'], row['Education'],row['Education-num'],row['Marital-status'],row['Occupation'],row['Relationship'],row['Race'],row['Sex'],row['Capital-gain'],row['Capital-loss'],row['Hours-per-week'],row['Native-country'],row['>50k, <=50k.']))

connection.commit()

In [15]:
#1. Select 10 records from the adult sqladb

 # connect to database
conn = sqlite3.connect("sqladb.db") 
result = pd.read_sql_query("select * from ADULTS LIMIT 10;", conn)
result

,Age,Workclass,Fnlwgt,Education,Education_num,Marital_status,Occupation,Relationship,Race,Sex,Capital_gain,Capital_loss,Hours_per_week,Native_country,">50k, <=50k."
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [24]:
# 2. Show me the average hours per week of all men who are working in private sector 
conn = sqlite3.connect("sqladb.db")
df_Mens_Workclass = pd.read_sql_query("select  Sex , Workclass , AVG(Hours_per_week)  from ADULTS where ltrim(upper(ADULTS.sex)) ='MALE' and ltrim(upper(ADULTS.WorkClass))='PRIVATE' group by Sex,WorkClass", conn)

df_Mens_Workclass

,Sex,Workclass,AVG(Hours_per_week)
0,Male,Private,42.221226


In [27]:
#3. Show me the frequency table for education, occupation and relationship, separately

conn = sqlite3.connect("sqladb.db")
eduction_df = pd.read_sql_query("SELECT Education, count(Education) FROM ADULTS GROUP BY Education;" , conn) # Query the database and convert data into dataframe

#Frequency table for education
eduction_df

,Education,count(Education)
0,10th,933
1,11th,1175
2,12th,433
3,1st-4th,168
4,5th-6th,333
5,7th-8th,646
6,9th,514
7,Assoc-acdm,1067
8,Assoc-voc,1382
9,Bachelors,5355


In [26]:
conn = sqlite3.connect("sqladb.db")
occupation_df  = pd.read_sql_query("SELECT Occupation, count(Occupation) FROM ADULTS GROUP BY Occupation;", conn)

#Frequency table for occupation
occupation_df

,Occupation,count(Occupation)
0,?,1843
1,Adm-clerical,3770
2,Armed-Forces,9
3,Craft-repair,4099
4,Exec-managerial,4066
5,Farming-fishing,994
6,Handlers-cleaners,1370
7,Machine-op-inspct,2002
8,Other-service,3295
9,Priv-house-serv,149


In [28]:
conn = sqlite3.connect("sqladb.db")
relationship_df  = pd.read_sql_query("SELECT Relationship, count(Relationship) FROM ADULTS GROUP BY Relationship;", conn) # Query the database and convert data into dataframe

#Frequency table for relationship
relationship_df

,Relationship,count(Relationship)
0,Husband,13193
1,Not-in-family,8305
2,Other-relative,981
3,Own-child,5068
4,Unmarried,3446
5,Wife,1568


In [31]:
#4. Are there any people who are married, working in private sector and having a masters degree

conn = sqlite3.connect("sqladb.db")
result  = pd.read_sql_query("SELECT workclass, education,'Marrid',count(*) from ADULTS a where ltrim(a.Marital_status) in('Married-civ-spouse','Married-spouse-absent','Married-AF-spouse') and ltrim(upper(a.Workclass))='PRIVATE' and  ltrim(upper(a.education))='MASTERS' group by a.workclass, a.education;", conn)

result

,Workclass,Education,'Marrid',count(*)
0,Private,Masters,Marrid,540


In [32]:
#5. What is the average, minimum and maximum age group for people working in different sectors

conn = sqlite3.connect("sqladb.db")
result  = pd.read_sql_query("SELECT Workclass, AVG(Age) , MIN(Age), MAX(Age) FROM ADULTS GROUP BY Workclass ;", conn)
result

,Workclass,AVG(Age),MIN(Age),MAX(Age)
0,?,40.960240,17,90
1,Federal-gov,42.590625,17,90
2,Local-gov,41.751075,17,90
3,Never-worked,20.571429,17,30
4,Private,36.797585,17,90
5,Self-emp-inc,46.017025,17,84
6,Self-emp-not-inc,44.969697,17,90
7,State-gov,39.436055,17,81
8,Without-pay,47.785714,19,72


In [38]:
#6. Calculate age distribution by country
conn = sqlite3.connect("sqladb.db")
result  = pd.read_sql_query("SELECT Native_country, Age , COUNT(Age) FROM ADULTS where ltrim(ADULTS.Native_country)<>'?' GROUP BY Native_country, Age ;", conn)
result.head(100)

,Native_country,Age,COUNT(Age)
0,Cambodia,18,1
1,Cambodia,25,1
2,Cambodia,27,2
3,Cambodia,28,1
4,Cambodia,32,1
5,Cambodia,34,1
6,Cambodia,35,1
7,Cambodia,36,1
8,Cambodia,37,2
9,Cambodia,40,2


In [39]:
#7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'

conn = sqlite3.connect("sqladb.db")
result  = pd.read_sql_query("SELECT *,(ADULTS.Capital_gain - ADULTS.Capital_loss) as 'Net-Capital-Gain' from ADULTS;", conn)
result.head(100)

,Age,Workclass,Fnlwgt,Education,Education_num,Marital_status,Occupation,Relationship,Race,Sex,Capital_gain,Capital_loss,Hours_per_week,Native_country,">50k, <=50k.",Net-Capital-Gain
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,2174
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,0
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K,0
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K,0
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K,0
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K,14084
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K,5178
